In [8]:
from cosmikyu import gan, config, model
from cosmikyu import nn as cnn
import numpy as np
import os
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import torch
import mlflow
import torchsummary

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
#STanh = cnn.ScaledTanh(5., 2./5.)
LF = cnn.LinearFeature(5,5, bias=True)
STanh = cnn.ScaledTanh(30., 2./30.)
Linear = torch.nn.Linear(5,5)
shape = (5,128,128)
FORSE = gan.FORSE_WGP("sehgal_forse_081020", shape, nconv_fcgen=64,
                          nconv_fcdis=64, cuda=False, ngpu=4, nconv_layer_gen=4, nconv_layer_disc=4, kernal_size=4, stride=2,
                          padding=1, output_padding=0, gen_act=[LF,STanh], nin_channel=5, nout_channel=5, nthresh_layer_gen=0, nthresh_layer_disc=0, dropout_rate=0)


print("====dc_wgp======")
#print(unet_gen)
#print(PIXGAN.generator)
torchsummary.summary(FORSE.generator, (5,128,128), batch_size=1, device="cpu")
torchsummary.summary(FORSE.discriminator, (5,128,128), batch_size=1, device="cpu")



Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/dwhan89/workspace/cosmikyu/cosmikyu/../output/mlruns/mlruns/meta.yaml' does not exist.


[WARNING] You have a CUDA device. You probably want to run with CUDA enabled
====dc_wgp======
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 64, 64, 64]           5,184
         LeakyReLU-2            [1, 64, 64, 64]               0
          UNetDown-3            [1, 64, 64, 64]               0
            Conv2d-4           [1, 128, 32, 32]         131,200
       BatchNorm2d-5           [1, 128, 32, 32]             256
         LeakyReLU-6           [1, 128, 32, 32]               0
          UNetDown-7           [1, 128, 32, 32]               0
            Conv2d-8           [1, 256, 16, 16]         524,544
       BatchNorm2d-9           [1, 256, 16, 16]             512
        LeakyReLU-10           [1, 256, 16, 16]               0
         UNetDown-11           [1, 256, 16, 16]               0
           Conv2d-12             [1, 512, 8, 8]       2,097,664
      Bat

In [ ]:
STanh = cnn.ScaledTanh(30., 2./30.)
LF = cnn.LinearFeature(4,4, bias=True)
shape = (5,128,128)
PIXGAN = gan.PIXGAN_WGP("sehgal_pixgan_251020", shape, nconv_fcgen=64,
                          nconv_fcdis=64, cuda=False, ngpu=4, nconv_layer_gen=4, nconv_layer_disc=5, kernal_size=4, stride=2,
                          padding=1, output_padding=0, gen_act=[LF,STanh], nin_channel=1, nout_channel=4, nthresh_layer_gen=3, nthresh_layer_disc=0, dropout_rate=0)


print("====dc_wgp======")
#print(unet_gen)
#print(PIXGAN.generator)
torchsummary.summary(PIXGAN.discriminator, (5,128,128), batch_size=1, device="cpu")
torchsummary.summary(PIXGAN.generator, (1,128,128), batch_size=1, device="cpu")

In [5]:
STanh = cnn.ScaledTanh(30., 2./30.)
LF = cnn.LinearFeature(5,5)
latent_dim = 256
shape = (5,128,128)

DCGAN_WGP = gan.DCGAN_WGP("sehgal_dcganwgp", shape, latent_dim, cuda=False, nconv_fcgen=64,
                              nconv_fcdis=64, ngpu=4, nconv_layer_gen=5, nconv_layer_disc=5, kernal_size=4, stride=2,
                                                        padding=1, output_padding=0, gen_act=[STanh, LF])

print("====cosmogan_wgp======")
torchsummary.summary(DCGAN_WGP.discriminator, shape, device="cpu")
torchsummary.summary(DCGAN_WGP.generator, (latent_dim,), device="cpu")
del DCGAN_WGP

Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/dwhan89/workspace/cosmikyu/cosmikyu/../output/mlruns/mlruns/meta.yaml' does not exist.


[WARNING] You have a CUDA device. You probably want to run with CUDA enabled
====cosmogan_wgp======
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           5,184
         LeakyReLU-2           [-1, 64, 64, 64]               0
            Conv2d-3          [-1, 128, 32, 32]         131,200
         LeakyReLU-4          [-1, 128, 32, 32]               0
            Conv2d-5          [-1, 256, 16, 16]         524,544
         LeakyReLU-6          [-1, 256, 16, 16]               0
            Conv2d-7            [-1, 512, 8, 8]       2,097,664
         LeakyReLU-8            [-1, 512, 8, 8]               0
            Conv2d-9           [-1, 1024, 4, 4]       8,389,632
        LeakyReLU-10           [-1, 1024, 4, 4]               0
          Reshape-11                [-1, 16384]               0
           Linear-12                    [-1, 1]          16,385
Tot

In [4]:


shape = (6,128,128)
genin_shape = (3,128,128)
discin_shape = (6,128,128)
unet_gen = model.UNET_Generator(
    genin_shape,
    nconv_layer=3,
    nconv_fc=64,
    ngpu=1,
    kernal_size=5,
    stride=2,
    padding=2,
    output_padding=1,
    normalize=True,
    activation=torch.nn.Tanh(),
    nin_channel = 3,
    nout_channel = 3,
    nthresh_layer = 4
).to(device="cpu")

unet_disc = model.UNET_Discriminator(
    shape=discin_shape,
    nconv_layer=4,
    nconv_fc=64,
    ngpu=1,
    kernal_size=5,
    stride=2,
    padding=2,
    normalize=False,
    nthresh_layer=2
).to(device="cpu")

print("====dc_wgp======")
torchsummary.summary(unet_disc, discin_shape, device="cpu")
torchsummary.summary(unet_gen, genin_shape, device="cpu")
#print(type(unet_disc.model))
#torchsummary.summary(DCGAN_WGP.generator, (latent_dim,))

====dc_wgp======
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,664
         LeakyReLU-2           [-1, 64, 64, 64]               0
            Conv2d-3          [-1, 128, 32, 32]         204,928
         LeakyReLU-4          [-1, 128, 32, 32]               0
            Conv2d-5          [-1, 256, 16, 16]         819,456
         LeakyReLU-6          [-1, 256, 16, 16]               0
            Conv2d-7            [-1, 512, 8, 8]       3,277,312
         LeakyReLU-8            [-1, 512, 8, 8]               0
            Conv2d-9            [-1, 512, 8, 8]       6,554,112
      BatchNorm2d-10            [-1, 512, 8, 8]           1,024
        LeakyReLU-11            [-1, 512, 8, 8]               0
           Conv2d-12            [-1, 512, 8, 8]       6,554,112
      BatchNorm2d-13            [-1, 512, 8, 8]           1,024
        LeakyReLU-14  

Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/dwhan89/workspace/cosmikyu/cosmikyu/../output/mlruns/mlruns/meta.yaml' does not exist.


====cosmogan_wgp======
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           8,064
         LeakyReLU-2           [-1, 64, 64, 64]               0
            Conv2d-3          [-1, 128, 32, 32]         204,928
         LeakyReLU-4          [-1, 128, 32, 32]               0
            Conv2d-5          [-1, 256, 16, 16]         819,456
         LeakyReLU-6          [-1, 256, 16, 16]               0
            Conv2d-7            [-1, 512, 8, 8]       3,277,312
         LeakyReLU-8            [-1, 512, 8, 8]               0
            Conv2d-9           [-1, 1024, 4, 4]      13,108,224
        LeakyReLU-10           [-1, 1024, 4, 4]               0
          Reshape-11                [-1, 16384]               0
           Linear-12                    [-1, 1]          16,385
           Conv2d-13           [-1, 64, 64, 64]           8,064
        LeakyReL

In [6]:
DCGAN_WGP.generator.__class__.__name__

/home/dwhan89/.miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'DCGAN_Generator'